In [47]:
#load dependencies
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

#read in data
lotto_df = pd.read_csv("../dailyData/dailyData.csv") 
lotto_df.head(10)

,Draw Date,Draw Schedule,First Number,Second Number,Third Number
0,7/9/2019,Morning,8,1,3
1,7/9/2019,Evening,2,3,0
2,7/10/2019,Morning,9,9,5
3,7/10/2019,Evening,2,3,2
4,7/11/2019,Morning,4,3,2
5,7/11/2019,Evening,3,6,2
6,7/12/2019,Morning,4,2,2
7,7/12/2019,Evening,5,1,0
8,7/13/2019,Morning,1,3,6
9,7/13/2019,Evening,1,2,4


In [26]:
lotto_df['lottoNum'] = lotto_df['First Number'].astype(str) +  lotto_df['Second Number'].astype(str) +  lotto_df['Third Number'].astype(str) 
lotto_df['lottoX'] = lotto_df['lottoNum'].shift(-1)
lotto_df = lotto_df.drop('Draw Date', axis=1)
lotto_df = lotto_df.drop('First Number', axis=1)
lotto_df = lotto_df.drop('Second Number', axis=1)
lotto_df = lotto_df.drop('Third Number', axis=1)
lotto_df = lotto_df.dropna()
lotto_df.head(10)

,Draw Schedule,lottoNum,lottoX
0,Morning,813,230
1,Evening,230,995
2,Morning,995,232
3,Evening,232,432
4,Morning,432,362
5,Evening,362,422
6,Morning,422,510
7,Evening,510,136
8,Morning,136,124
9,Evening,124,954


In [28]:
lotto_df['lottoNum'] = lotto_df['lottoNum'].astype(int)
lotto_df['lottoX'] = lotto_df['lottoX'].astype(int)

lotto_df.dtypes

Draw Schedule    object
lottoNum          int64
lottoX            int64
dtype: object

In [29]:
split_df = pd.get_dummies(lotto_df)
split_df.head(10)

,lottoNum,lottoX,Draw Schedule_Evening,Draw Schedule_Morning
0,813,230,0,1
1,230,995,1,0
2,995,232,0,1
3,232,432,1,0
4,432,362,0,1
5,362,422,1,0
6,422,510,0,1
7,510,136,1,0
8,136,124,0,1
9,124,954,1,0


In [59]:
X = split_df[['lottoX','Draw Schedule_Evening', 'Draw Schedule_Morning']]
y = split_df['lottoNum'].values.reshape(-1, 1)
print(X.shape,y.shape)

(221, 3) (221, 1)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [61]:
# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [62]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [63]:
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = ridge.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = ridge.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.0727402207453178, R2: -0.029386889180413656


In [41]:
import pickle
#save model
pkl_filename = "../MLModels/SKLearn_Ridge_MornEve.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(ridge, file)

In [68]:
# Load saved model
pkl_filename = "../MLModels/SKLearn_Ridge_MornEve.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_ridge = pickle.load(file)

In [69]:
# predict target values
RidgePredict = pickle_ridge.predict([[781,1,0]])
RidgePredict

array([[91.971469]])

In [70]:
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = lasso.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.068778253717858, R2: -0.025585039641670138


In [72]:
#save model
pkl_filename2 = "../MLModels/SKLearn_Lasso_MornEve.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lasso, file)

In [73]:
# Load saved model
pkl_filename2 = "../MLModels/SKLearn_Lasso_MornEve.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_lasso = pickle.load(file)

In [74]:
# predict target values
LassoPredict = pickle_lasso.predict([[781,1,0]])
LassoPredict

array([83.93180985])

In [75]:
elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = elasticnet.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.0705160798005167, R2: -0.02725263385559984


In [77]:
#save model
pkl_filename3 = "../MLModels/SKLearn_ElasticNet_MornEve.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(elasticnet, file)

In [78]:
# Load saved model
pkl_filename2 = "../MLModels/SKLearn_ElasticNet_MornEve.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_elasticnet = pickle.load(file)

In [79]:
# predict target values
ElasticNetPredict = pickle_elasticnet.predict([[781,1,0]])
ElasticNetPredict

array([87.52404004])